# Описание блокнота

Данный блокнот служит для наглядного тестирования модуля взаимодействия с исходными Excel-файлами.

# 0. Импорт библиотек

In [1]:
# !pip freeze > jupiter-requirements.txt

In [2]:
# Импорты парсера
import cr_component.parser.additional as cr_add    # Хранилище побочных функций
import cr_component.parser.defaults   as cr_def    # Константы парсера
import cr_component.parser.reader     as cr_read   # Взаимодействие с Excel
import cr_component.parser.parser     as cr_parse  # Регулярный парсер
import cr_component.parser.writer     as cr_write  # Форматная запись в Excel

In [3]:
# Общие импорты
import os
import re

In [4]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', 905)

In [5]:
from IPython.core.interactiveshell import InteractiveShell
jup_conds = ['all',                # Показывать все вызовы
             'last',               # Показывать последнюю общность вызовов
             'last_expr',          # Показывать самый последний вызов
             'none',               # Ничего не показывать
             'last_expr_or_assign' # Показывать последний вызов или присваивание
            ]

InteractiveShell.ast_node_interactivity = jup_conds[0]

In [6]:
base_dir = os.getcwd()
path_load = os.path.join(base_dir, 'Tests', 'Datasets')  # Путь загрузки обрабатываемых файлов
path_save = os.path.join(base_dir, 'Tests', 'Results')   # Путь сохранения итогов обработки
path_json = os.path.join(base_dir, 'Tests', 'Jsons')     # Путь сохранения json файлов
path_best = os.path.join(base_dir, 'Tests', 'JSBest')    # Где хранятся проверенные вручную json файлы

# 1. Тестирование модуля взаимодействия с исходным Excel-документом

In [7]:
help(cr_read)

Help on module cr_component.parser.reader in cr_component.parser:

NAME
    cr_component.parser.reader

DESCRIPTION
    Взаимодействие с исходным Excel-документом.
    Данный модуль предназначен для:
        - Считывания изначального файла расписания из .xls или .xlsx;
        - Вычленения расписания конкретной выбранной группы из считанного файла.

FUNCTIONS
    get_cell_value(row: 'Координата строки', col: 'Координата столбца', mgl: 'Список границ объединённых ячеек', act: 'Нужна актуальная ячейка, или её правый сосед?') -> 'Координаты ячейки, из которой нужно взять значение'
        Функция получения координат истинного значения ячейки (в том числе, объединённой)
    
    group_choice(sheet: 'Объект листа с загруженной книги') -> 'Словарь {Список групп на листе, индексный диапазон расписания}'
        Функция для выделения описательной информации о группах и границах на листе
    
    prepare(sheet: 'Выбранный лист', group: 'Выбранная группа', coord: 'Диапазон информации о расписани

In [14]:
def test_reader(path, file):
    """ Функция для тестирования взаимодействия с исходными файлами расписания """
    
    # Построение пути к заданному файлу в заданной директории
    file_path = os.path.join(path, file)
    print(f'Открытие файла «{file_path}»')

    # Считывание EXCEL-таблицы
    book = cr_read.read_book(file_path)
    
    # Получение списка названий листов в книге
    sheets = cr_read.see_sheets(book)
    print(f'Список листов в файле: «{sheets}»')

    # Проход по всем названиям листов в книге
    for sheet_name in sheets:
        # Выбор листа sheet_name
        if file == '1_KURS_2018-2019_2semestr.xls':
            sheet_name = 'aaaaaaa'
        sheet = cr_read.take_sheet(book, sheet_name)
        # Получение информации о выбранном листе
        sheet_info = cr_read.group_choice(sheet)
        print(f'{sheet_name}: {sheet_info}')
        
        period, year, groups, start_end = sheet_info.values()
        for group in groups:
            # print(f'Формирование предобработанных данных для группы «{group}»')
            bd_process = cr_read.prepare(sheet, group, start_end)

In [15]:
# Проход по всем файлам и подпапкам в директории
for root, d, files in os.walk(path_load):
    for file in files:
        try:
            test_reader(path_load, file)
        except Exception as err:
            print(err)
        print()

Открытие файла «E:\CoolResp\Tests\Datasets\0NewEvolution_0_62_8.w3x»
Не удалось открыть файл как Excel-книгу!

Открытие файла «E:\CoolResp\Tests\Datasets\0Респа для ПЕ-81б на [25.01.2022 - 13.04.2022].xlsx»
Список листов в файле: «['Январь', 'Февраль', 'Апрель']»
Невозможно выявить содержательную часть расписания!
На листе не обнаружены позиционные метки: <ссылка на позиционный образец>

Открытие файла «E:\CoolResp\Tests\Datasets\1_KURS_2018-2019_2semestr.xls»
Список листов в файле: «['МЕ, ИТ, ОЕ81б']»
No sheet named <'aaaaaaa'>

Открытие файла «E:\CoolResp\Tests\Datasets\2_kurs_2semestr_2018-2019.xls»
Список листов в файле: «['МЕ,ОЕ 71б', 'ИТ,ВЕ-71б']»
МЕ,ОЕ 71б: {'period': ['28.01.19', '01.06.19'], 'year': 2019, 'groups_info': ['МЕ-71б', 'ОЕ-71б'], 'range': (10, 51)}
ИТ,ВЕ-71б: {'period': ['28.01.19', '01.06.19'], 'year': 2019, 'groups_info': ['ИТ-71б', 'ВЕ-71б'], 'range': (10, 48)}

Открытие файла «E:\CoolResp\Tests\Datasets\3_kurs_2018-2019_II_semestr.xls»
Список листов в файле: «[